In [1]:
import cv2
from cardshark.utils import count_points
from cardshark import Vision
from time import time
import pandas as pd

In [2]:
file = pd.read_csv('db.csv', sep=';')

paths = [row[1]['file_path'] for row in file.iterrows()]
pointss = [row[1]['labels'] for row in file.iterrows()]

expected_pointss = [list(map(int, points.split(','))) for points in pointss]

In [3]:
for scale in [1.0, 0.75, 0.5, 0.25]:
    print(f'For scale {scale}:')
    _all = 0
    _all_positions = 0
    _correct = 0
    _correct_positions = 0
    
    start = time()
    for path, expected_points in zip(paths, expected_pointss):
        vision = Vision(path, scale=scale)
        positions_and_points = [(cv2.boundingRect(contour)[:2], count_points(contents)) for contour, contents in zip(vision.pile_contours, vision.piles_contents)]
        
        positions_and_points.sort(key=lambda pos_and_point: pos_and_point[0][1])
        croupier_points = [positions_and_points[0][1]]
        
        positions_and_points = positions_and_points[1:]
        positions_and_points.sort(key=lambda pos_and_point: pos_and_point[0][0])
        
        players_points = [pos_and_point[1] for pos_and_point in positions_and_points]
        
        points = croupier_points + players_points
        
        _all += 1
        _all_positions += len(points)            
        if points == expected_points:
            _correct += 1
        # else:
        #     print(f'Path: {path}, expected: {expected_points} but got: {points}')
        _correct_positions += [p == ep for p, ep in zip(points, expected_points)].count(True)
        
    total_time = time() - start
    
    
    print(f"Accuracy (exact):      {_correct/_all * 100:.3f} %    Time: {total_time:.3f} s  Frames: {len(paths)}")
    print(f"Accuracy (positional): {_correct_positions/_all_positions * 100:.3f} %    Average Time per Frame: {total_time / len(paths):.3f} s")
    print()
    print()

For scale 1.0:
Accuracy (exact):      100.000 %    Time: 19.871 s  Frames: 64
Accuracy (positional): 100.000 %    Average Time per Frame: 0.310 s


For scale 0.75:
Accuracy (exact):      100.000 %    Time: 18.085 s  Frames: 64
Accuracy (positional): 100.000 %    Average Time per Frame: 0.283 s


For scale 0.5:
Accuracy (exact):      96.875 %    Time: 14.724 s  Frames: 64
Accuracy (positional): 99.340 %    Average Time per Frame: 0.230 s


For scale 0.25:
Accuracy (exact):      43.750 %    Time: 13.184 s  Frames: 64
Accuracy (positional): 79.538 %    Average Time per Frame: 0.206 s

